In [ ]:
from easydict import EasyDict as edict
cfg = {
     "train_csv": "config/train.csv",
    "dev_csv": "config/dev.csv",
    "backbone": "densenet121",
    "width": 320,
    "height": 320,
    "long_side": 320,
    "fix_ratio": True,
    "pixel_mean": 128.0,
    "pixel_std": 64.0,
    "use_pixel_std": True,
    "use_equalizeHist": True,
    "use_transforms_type": "Aug",
    "gaussian_blur": 3,
    "border_pad": "pixel_mean",
    "num_classes": [1,1,1,1,1],
    "batch_weight": True,
    "enhance_index": [2,6],
    "enhance_times": 1,
    "pos_weight": [1,1,1,1,1],
    "train_batch_size": 16,
    "dev_batch_size": 1,
    "pretrained": True,
    "log_every": 50,
    "test_every": 200,
    "epoch": 30,
    "norm_type": "BatchNorm",
    "global_pool": "PCAM",
    "fc_bn": False,
    "attention_map": "None",
    "lse_gamma": 0.5,
    "fc_drop": 0,
    "optimizer": "Adam",
    "criterion": "BCE",
    "lr": 0.001,
    "lr_factor": 0.1,
    "lr_epochs": [2],
    "momentum": 0.9,
    "weight_decay": 0.0,
    "best_target": "auc",
    "save_top_k": 30,
    "save_index": [0,1,2,3,4],
    'save_path' : '/content/drive/MyDrive/Biomed/efficient net',
    'logtofile' : True,
    'resume' : False,
    'pre_train' : None,
    'verbose' :  False
}
cfg  = edict(cfg)

In [ ]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
from collections import OrderedDict
from utils import get_norm

__all__ = ['DenseNet', 'densenet121', 'densenet169', 'densenet201', 'densenet161']  # noqa


model_urls = {
    'densenet121': 'https://download.pytorch.org/models/densenet121-a639ec97.pth',  # noqa
    'densenet169': 'https://download.pytorch.org/models/densenet169-b2777c0a.pth',  # noqa
    'densenet201': 'https://download.pytorch.org/models/densenet201-c1103571.pth',  # noqa
    'densenet161': 'https://download.pytorch.org/models/densenet161-8d451a50.pth',  # noqa
}


class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate,
                 norm_type='Unknown'):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', get_norm(norm_type, num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size *
                        growth_rate, kernel_size=1, stride=1, bias=False)),
        self.add_module('norm2', get_norm(norm_type, bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                        kernel_size=3, stride=1, padding=1, bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)  # noqa
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate, norm_type='Unknown'):  # noqa
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate, norm_type=norm_type)  # noqa
            self.add_module('denselayer%d' % (i + 1), layer)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features,
                 norm_type='Unknown'):
        super(_Transition, self).__init__()
        self.add_module('norm', get_norm(norm_type, num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,  # noqa
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))


class DenseNet(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_  # noqa

    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer  # noqa
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
    """

    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 norm_type='Unknown', num_init_features=64, bn_size=4, drop_rate=0, num_classes=1000):  # noqa

        super(DenseNet, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(3, num_init_features, kernel_size=7, stride=2, padding=3, bias=False)),  # noqa
            ('norm0', get_norm(norm_type, num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features, norm_type=norm_type,  # noqa
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)  # noqa
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2, norm_type=norm_type)  # noqa
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', get_norm(norm_type, num_features))

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)
        self.num_features = num_features

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.GroupNorm):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.InstanceNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x_pre = x
        for i in  self.features :
            x = i(x)
            if x_pre.size(2) > x.size(2):
                #print(i)
                print(x.shape)
                x_pre = x
        out = F.relu(x, inplace=True)
        # out = F.adaptive_avg_pool2d(out, (1, 1)).view(features.size(0), -1)
        # out = self.classifier(out)
        return out


def densenet121(cfg, **kwargs):
    r"""Densenet-121 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_  # noqa

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = DenseNet(num_init_features=64, growth_rate=32, block_config=(6, 12, 24, 16),  # noqa
                     norm_type=cfg.norm_type, **kwargs)
    if cfg.pretrained:
        # '.'s are no longer allowed in module names, but pervious _DenseLayer
        # has keys 'norm.1', 'relu.1', 'conv.1', 'norm.2', 'relu.2', 'conv.2'.
        # They are also in the checkpoints in model_urls. This pattern is used
        # to find such keys.
        pattern = re.compile(
            r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')  # noqa
        state_dict = model_zoo.load_url(model_urls['densenet121'])
        for key in list(state_dict.keys()):
            res = pattern.match(key)
            if res:
                new_key = res.group(1) + res.group(2)
                state_dict[new_key] = state_dict[key]
                del state_dict[key]
        model.load_state_dict(state_dict, strict=False)
    return model


def densenet169(cfg, **kwargs):
    r"""Densenet-169 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_  # noqa

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = DenseNet(num_init_features=64, growth_rate=32, block_config=(6, 12, 32, 32),  # noqa
                     norm_type=cfg.norm_type, **kwargs)
    if cfg.pretrained:
        # '.'s are no longer allowed in module names, but pervious _DenseLayer
        # has keys 'norm.1', 'relu.1', 'conv.1', 'norm.2', 'relu.2', 'conv.2'.
        # They are also in the checkpoints in model_urls. This pattern is used
        # to find such keys.
        pattern = re.compile(
            r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')  # noqa
        state_dict = model_zoo.load_url(model_urls['densenet169'])
        for key in list(state_dict.keys()):
            res = pattern.match(key)
            if res:
                new_key = res.group(1) + res.group(2)
                state_dict[new_key] = state_dict[key]
                del state_dict[key]
        model.load_state_dict(state_dict, strict=False)
    return model


def densenet201(cfg, **kwargs):
    r"""Densenet-201 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_  # noqa

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = DenseNet(num_init_features=64, growth_rate=32, block_config=(6, 12, 48, 32),  # noqa
                     norm_type=cfg.norm_type, **kwargs)
    if cfg.pretrained:
        # '.'s are no longer allowed in module names, but pervious _DenseLayer
        # has keys 'norm.1', 'relu.1', 'conv.1', 'norm.2', 'relu.2', 'conv.2'.
        # They are also in the checkpoints in model_urls. This pattern is used
        # to find such keys.
        pattern = re.compile(
            r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')  # noqa
        state_dict = model_zoo.load_url(model_urls['densenet201'])
        for key in list(state_dict.keys()):
            res = pattern.match(key)
            if res:
                new_key = res.group(1) + res.group(2)
                state_dict[new_key] = state_dict[key]
                del state_dict[key]
        model.load_state_dict(state_dict, strict=False)
    return model


def densenet161(cfg, **kwargs):
    r"""Densenet-161 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_  # noqa

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = DenseNet(num_init_features=96, growth_rate=48, block_config=(6, 12, 36, 24),  # noqa
                     norm_type=cfg.norm_type, **kwargs)
    if cfg.pretrained:
        # '.'s are no longer allowed in module names, but pervious _DenseLayer
        # has keys 'norm.1', 'relu.1', 'conv.1', 'norm.2', 'relu.2', 'conv.2'.
        # They are also in the checkpoints in model_urls. This pattern is used
        # to find such keys.
        pattern = re.compile(
            r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')  # noqa
        state_dict = model_zoo.load_url(model_urls['densenet161'])
        for key in list(state_dict.keys()):
            res = pattern.match(key)
            if res:
                new_key = res.group(1) + res.group(2)
                state_dict[new_key] = state_dict[key]
                del state_dict[key]
        model.load_state_dict(state_dict, strict=False)
    return model

In [ ]:
model = densenet121(cfg)

NameError: name 'densenet121' is not defined

In [6]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [20]:
all_layers = []

for layer in model.modules():
    if type(layer) == nn.Sequential: # if sequential layer, apply recursively to layers in sequential layer
        remove_sequential(layer)
    if list(layer.children()) == []: # if leaf node, add it to list
        all_layers.append(layer)

In [88]:
module_list = []
def remove_sequential(model,module_list) :
  
    for i in model.children() :
        if isinstance(i, nn.Sequential) :
            for j   in  i:
                module_list.append(j)
        else :
            module_list.append(i)
    return module_list

In [104]:
features = list(model.children())[0]
layers = []

In [105]:
 for i in features :
        layers.append(i)

In [107]:
x = torch.rand(3,3,320,320)
for i in layers :
    x  = i(x)


In [91]:
x = torch.rand((3,3,320,320))
for i in l :
    #print(i)
    x = i(x)
    

RuntimeError: mat1 and mat2 shapes cannot be multiplied (30720x10 and 1024x1000)

In [ ]:
for i in model.children():
    print(i)

In [49]:
# class MyModule(nn.Module):
#     def __init__(self,l):
#         super().__init__()
#         self.layers = nn.ModuleList(l)
#         self.trace = []
        
#     def forward(self,x):
#         for layer in self.layers:
#             x = layer(x)
            
#         return x

In [64]:
x = torch.rand(3,3,320,320)
x_pre  = x
for i in  model.features :
    x = i(x)
    if x_pre.size(2) > x.size(2):
        print(i)
        print(x.shape)
        x_pre = x
    

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
torch.Size([3, 64, 160, 160])
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
torch.Size([3, 64, 80, 80])
_Transition(
  (norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
)
torch.Size([3, 128, 40, 40])
_Transition(
  (norm): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
)
torch.Size([3, 256, 20, 20])
_Transition(
  (norm): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=

In [ ]:
model(torch.rand(3,3,320,320))